# Expanding on 01_dataset_generation

This notebook is for expanding on [01_dataset_generation](/01_dataset_generation.ipynb), where we generated our initial dataset. After completing our first notebook we found more datasets that may be useful in developing our UHI model. Again, we may or may not add additional variables later in our pipeline

## Dataset Description

Please relate to our [first notebook](/01_dataset_generation.ipynb) to find a description on our other datasets. This notebook will only explain our new datasets we found.

### Feature Datasets:

- **Building Footprint Data with Additional Attributes**
   - Data from NYC Office of Technology and Innovation (OTI)
   - Useful for understanding the impact of buildings on local temperatures.
   - This dataset has more attributes than **Building Footprint Data** such as Building height.

- **Automated Traffic Volume Counts**
   -  Data from New York City Department of Transportation (NYC DOT)
   - Useful for understanding the impact of traffic on local temperatures.

>NOTE: This is a developer note, **TODO**: Remember to minus the ground level from the building height. This will normalize all building so they compared from a equal ground level.

## Combining The Data

### Set Up

Download libraries not in google collab (can be disregarded if not using collab)

In [ ]:
%pip install stackstac
%pip install pystac-client
%pip install planetary-computer
%pip install odc-stac
%pip install rioxarray

Import libraries

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import RdYlGn,jet,RdBu
from google.colab import drive
import geopandas as gpd
from shapely.geometry import Point, Polygon
from rasterio.mask import mask
from shapely.ops import transform
import pyproj
import shapely.vectorized
from geopy.distance import geodesic

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

#other libs
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
from datetime import timedelta

Mount google drive, ignore if not using Google Collab:

In [ ]:
# mount google drive
drive.mount('/content/drive')
%cd /content/drive/My Drive
!git clone https://github.com/FranciscoLozCoding/cooling_with_code.git
%cd cooling_with_code
!git pull

### Helper Functions

### Discover and load the data for analysis